<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Imports-and-Constants" data-toc-modified-id="Imports-and-Constants-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Imports and Constants</a></span></li><li><span><a href="#General-Helper-Functions" data-toc-modified-id="General-Helper-Functions-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>General Helper Functions</a></span></li><li><span><a href="#DHS-Helper-Functions" data-toc-modified-id="DHS-Helper-Functions-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>DHS Helper Functions</a></span></li><li><span><a href="#DHS-NL" data-toc-modified-id="DHS-NL-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>DHS NL</a></span></li><li><span><a href="#DHS-MS" data-toc-modified-id="DHS-MS-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>DHS MS</a></span></li><li><span><a href="#LSMS-indexofdelta-MS" data-toc-modified-id="LSMS-indexofdelta-MS-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>LSMS indexofdelta MS</a></span></li></ul></div>

This notebook generates activation maps for different models. Generating the activation maps should be done on a machine with a GPU. However, once the raw activation maps have been pickled, viewing them can be done on CPU-only.

# Imports and Constants

In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
from collections import defaultdict
import heapq
import os
import pickle
import sys
import time

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import PIL
import tensorflow as tf

sys.path.append('../')
from batchers import batcher, dataset_constants, delta_batcher
from models.resnet_model import Hyperspectral_Resnet
from utils.general import add_to_heap

In [ ]:
CKPTS_DIR = '../ckpts'
LOGS_DIR = '../logs'
NUM_TOP_IMGS = 8

MODEL_PATHS = {
    'incountry_resnet_ms_D': [
        'DHSIncountry/incountryD_18preact_ms_samescaled_b64_fc001_conv001_lr0001',
        'ckpt-40'],
    'incountry_resnet_nl_C': [
        'DHSIncountry/incountryC_18preact_nl_random_b64_fc1.0_conv1.0_lr0001',
        'ckpt-145'],
    'lsms_indexofdelta_incountry_ms_D': [
        'LSMSIndexOfDeltaIncountry/LSMSIndexOfDeltaIncountryD_bidir_18preact_ms_random_b64_fc1_conv1_lr0001',
        'ckpt-64']
}

BATCH_SIZE = 128
LABEL_NAME = 'wealthpooled'
GPU = 0
GPU_USAGE = 0.9
IS_TRAINING = False
MODEL_PARAMS = {
    'fc_reg': 5e-3,
    'conv_reg': 5e-3,
    'num_layers': 18,
    'num_outputs': 1
}

TENSOR_NAMES = {
    'scale1_img': 'resnet/scale1/scale1_img:0',  # batch_size, 112, 112,  64
    'scale2_img': 'resnet/scale2/scale2_img:0',  # batch_size,  56,  56,  64
    'scale3_img': 'resnet/scale3/scale3_img:0',  # batch_size,  28,  28, 128
    'scale4_img': 'resnet/scale4/scale4_img:0',  # batch_size,  14,  14, 256
}

# General Helper Functions

In [ ]:
def smart_img_rescale(image):
    '''
    Args
    - image: np.array, shape [H, W] or [H, W, 3]

    Returns: img, np.array, copy of image, rescaled, then clipped at [0, 1]
    '''
    img = np.array(image)
    # center images towards mean-0
    img = img / 6
    mean = np.mean(img)
    img -= np.sign(mean) * min(0.9*abs(mean), abs(mean)**1.4)
    new_mean = np.mean(img)

    # scale images towards std-dev 1/6
    std = np.std(img)
    img = (img - new_mean) * (0.16 / std)**0.7 + new_mean
    print('Mean_0:', mean, 'Mean_new:', new_mean, 'Std_0:', std, 'Std_new:', np.std(img))

    img = np.clip(img + 0.5, a_min=0, a_max=1)
    return img

In [ ]:
def images_grid(imgs, size, resize=PIL.Image.NEAREST, spacing=0, color=0):
    '''
    Args
    - imgs: list of list of PIL.Image, 1 sublist for each row of images
    - size: int, width/height in pixels to reshape images to
    - resize: PIL filter
    - spacing: int, number of pixels between adjacent images
    - color: int, spacing color, 0 for black, 255 for white

    Returns: np.array, grid of images
    '''
    nrows = len(imgs)
    ncols = len(imgs[0])

    gridH = nrows * size + (nrows - 1) * spacing
    gridW = ncols * size + (ncols - 1) * spacing
    grid = np.ones([gridH, gridW, 3], dtype=np.uint8) * color

    for r in range(nrows):
        for c in range(ncols):
            img = imgs[r][c]
            if img.size != (size, size):
                img = img.resize((size, size), resize)
            i = r * (size + spacing)
            j = c * (size + spacing)
            grid[i:i+size, j:j+size, :] = np.asarray(img).reshape(size, size, -1)
    return grid

# DHS Helper Functions

In [ ]:
def get_dhs_batcher(ls_bands, nl_band):
    '''
    Args
    - ls_bands: one of ['rgb', 'ms', None]
    - nl_band: one of ['merge', 'split', None]

    Returns
    - batcher: Batcher
    '''
    all_tfrecord_paths = np.asarray(batcher.get_tfrecord_paths('2009-17', 'all'))
    assert len(all_tfrecord_paths) == dataset_constants.SIZES['2009-17']['all']

    b = batcher.Batcher(
        tfrecord_files=all_tfrecord_paths,
        dataset='2009-17',
        batch_size=BATCH_SIZE,
        label_name=LABEL_NAME,
        num_threads=4,
        epochs=1,
        ls_bands=ls_bands,
        nl_band=nl_band,
        shuffle=False,
        augment=False,
        normalize=True,
        cache=False)
    return b

In [ ]:
def run_batches(sess, tensors_dict_ops, tensor_name, max_nbatches=None, verbose=False):
    '''Runs the ops in tensors_dict_ops for a fixed number of batches or until
    reaching a tf.errors.OutOfRangeError, concatenating the runs.

    Note: assumes that the dataset iterator doesn't need initialization, or is
        already initialized.

    Args
    - sess: tf.Session
    - tensors_dict_ops: dict, str => tf.Tensor, shape [batch_size] or [batch_size, D]
      - keys: ['images', 'years', 'locs', tensor_name]
    - tensor_name: str, key in TENSOR_NAMES
    - max_nbatches: int, maximum number of batches to run the ops for,
        set to None to run until reaching a tf.errors.OutOfRangeError
    - verbose: bool, whether to output batch processing progress

    Returns
    - top_images_avg: dict, maps filter number (int) into list of (value, data) tuples
        value = (mean filter activation for a particular image, negative image index)
        data = (img, year, loc, activation map)
    '''
    top_images_avg = defaultdict(list)

    curr_batch = 0
    start_time = time.time()
    try:
        while True:
            all_tensors = sess.run(tensors_dict_ops)

            imgs = all_tensors['images']  # N, H, W, C
            years = all_tensors['years']
            locs = all_tensors['locs']
            actmaps = all_tensors[tensor_name]

            batch_size, _, _, num_filters = actmaps.shape

            actmaps_means = np.mean(actmaps, axis=(1, 2), dtype=np.float64)  # shape [batch_size, num_filters]

            for i in range(batch_size):
                img = imgs[i]
                year = years[i]
                loc = tuple(locs[i])
                actmap = actmaps[i]  # shape [H, W, num_filters]
                actmap_means = actmaps_means[i]  # shape [num_filters]

                for f in range(num_filters):
                    value = (actmap_means[f], -(i + curr_batch*BATCH_SIZE))
                    data = (img, year, loc, actmap[:, :, f])
                    add_to_heap(h=top_images_avg[f], k=NUM_TOP_IMGS, value=value, data=data)

            curr_batch += 1
            if verbose:
                speed = curr_batch / (time.time() - start_time)
                print(f'\rRan {curr_batch} batches ({speed:.3f} batch/s)', end='')
            if (max_nbatches is not None) and (curr_batch >= max_nbatches):
                break
            if curr_batch >= 15:
                break

    except tf.errors.OutOfRangeError:
        pass

    print()  # print a newline, since the previous print()'s don't print newlines
    return top_images_avg

In [ ]:
def get_max_act_images(ckpt_path: str, fold: str, ls_bands: str,
                       nl_band: str, tensor_name: str):
    '''
    Args
    - ckpt_path: str
    - fold: str, one of ['A', 'B', 'C', 'D', 'E']
    - ls_bands: str or None
    - nl_band: str or None
    - tensor_name: str, key of TENSOR_NAMES

    Returns
    - top_images_avg: dict, maps filter number (int) into list of (value, data) tuples
        value = (mean filter activation for a particular image, negative image index)
        data = (img, year, loc, activation map)
    '''
    tf.reset_default_graph()

    print('=== Running model ===')
    print('- ckpt:', ckpt_path)
    print('- fold:', fold)
    print('- ls_bands, nl_band:', ls_bands, nl_band)

    init_iter, batch_op = get_dhs_batcher(ls_bands, nl_band).get_batch()

    print('Building model...')
    model = Hyperspectral_Resnet(batch_op['images'], is_training=IS_TRAINING, **MODEL_PARAMS)

    graph = tf.get_default_graph()
    tensors_dict_ops = {
        'images': batch_op['images'],
        'years': batch_op['years'],
        'locs': batch_op['locs'],
        tensor_name: graph.get_tensor_by_name(TENSOR_NAMES[tensor_name])
    }

    print('Creating session...')
    os.environ['CUDA_VISIBLE_DEVICES'] = str(GPU)
    config_proto = tf.ConfigProto()
    config_proto.gpu_options.per_process_gpu_memory_fraction = GPU_USAGE

    with tf.Session(config=config_proto) as sess:
        sess.run(init_iter)

        # clear the model weights, then load saved checkpoint
        print('Loading saved ckpt...')
        saver = tf.train.Saver(var_list=None)
        sess.run([tf.global_variables_initializer(), tf.local_variables_initializer()])
        saver.restore(sess, ckpt_path)

        # run the saved model
        top_images_avg = run_batches(sess, tensors_dict_ops, tensor_name, verbose=True)
    return top_images_avg

In [ ]:
def plot_activations(imgs, actmaps, locs=None, years=None, title=None, size=4, nl=False, savedir=None):
    '''
    Args
    - imgs: list of np.array, length N, each np.array has shape [H, W] or [H, W, 3]
        - if [H, W, 3], then band order is assumed to be R, G, B
    - actmaps: list of np.array, length N, each np.array has shape [actH, actW]
    - locs: list of (lat, lon) tuples
    - years: list of int
    - title: str, figure title
    - size: int, size of each img, in inches
    - nl: bool, when plotting nightlights images
    - savedir: str, path to directory to save imgs and activation maps
    '''
    nimgs = len(imgs)
    assert len(actmaps) == nimgs

    # make copy, so we don't modify the originals
    np_imgs = np.array(imgs)
    np_actmaps = np.abs(np.array(actmaps))  # activation maps aren't always after a ReLU

    # sort images by mean activation in descending order
    sorted_index = np.argsort(np.mean(np_actmaps, axis=(1, 2)))[::-1]
    np_imgs = np_imgs[sorted_index]
    np_actmaps = np_actmaps[sorted_index]

    if locs is not None:
        locs = np.array(locs)  # make copy
        locs = locs[sorted_index]
    if years is not None:
        years = np.array(years)  # make copy
        years = years[sorted_index]

    max_act = np.percentile(np_actmaps, q=99)

    fig, axs = plt.subplots(2, nimgs, figsize=[size*nimgs, size*2])

    for i in range(nimgs):
        img = smart_img_rescale(np_imgs[i])
        actmap = np_actmaps[i]

        if nl:
            mean = np.mean(actmap)
            std = np.std(actmap)
            actmap_max = min(np.max(actmap), mean + 6 * std)
            actmap_min = max(np.min(actmap), mean - 6 * std)
            actmap = (actmap - actmap_min) / actmap_max
            actmap = np.clip(actmap, a_min=0, a_max=1)
        else:
            actmap = np.clip(actmap, a_min=0, a_max=max_act) / max_act
    
        # origin='lower' to match lat/lon direction
        axs[0, i].imshow(img, origin='lower', vmin=0, vmax=1)
        axs[1, i].imshow(actmap, origin='lower', vmin=0, vmax=1,
                         interpolation='none', cmap='gray')

        axs[0, i].axis('off')
        axs[1, i].axis('off')

        ax_title = []
        if locs is not None:
            lat, lon = locs[i]
            ax_title.append(f'loc: ({lat:.4f}, {lon:.4f})')
        if years is not None:
            year = years[i]
            ax_title.append(f'year: {year}')
        if len(ax_title) > 0:
            ax_title = ' '.join(ax_title)
            axs[0, i].set_title(ax_title)

        if savedir is not None:
            os.makedirs(savedir, exist_ok=True)
            img_filename = os.path.join(savedir, f'img_{i}.png')
            # plt.imsave(img_filename, img, vmin=0, vmax=1, format='png', origin='lower')
            print('Saving image to', img_filename)
            PIL.Image.fromarray((img * 255).astype(np.uint8)).transpose(PIL.Image.FLIP_TOP_BOTTOM).save(img_filename, optimize=True)

            actmap_filename = os.path.join(savedir, f'actmap_{i}.png')
            # plt.imsave(actmap_filename, actmap, vmin=0, vmax=1, format='png', origin='lower', cmap='gray')
            print('Saving activation map to', actmap_filename)
            PIL.Image.fromarray((actmap * 255).astype(np.uint8)).transpose(PIL.Image.FLIP_TOP_BOTTOM).save(actmap_filename, optimize=True)

    if title is not None:
        fig.suptitle(title, y=1.03)
    fig.subplots_adjust(wspace=0, hspace=0)
    fig.tight_layout()
    plt.show()

# DHS NL

In [ ]:
actmaps_pkl_path = 'dhs_incountryC_actmaps_nl.pkl'
if not os.path.exists(actmaps_pkl_path):
    ckpt_path = os.path.join(CKPTS_DIR, *MODEL_PATHS['incountry_resnet_nl_C'])
    fold = 'C'
    ls_bands = None
    nl_band = 'split'
    tensor_name = 'scale2_img'

    top_images_avg_nl = get_max_act_images(ckpt_path, fold, ls_bands, nl_band, tensor_name)
    with open(actmaps_pkl_path, 'wb') as f:
        pickle.dump(top_images_avg_nl, f)

In [ ]:
with open(actmaps_pkl_path, 'rb') as f:
    top_images_avg_nl = pickle.load(f)

In [ ]:
for f in range(len(top_images_avg_nl)):
    imgs = []
    locs = []
    actmaps = []

    for value, data in top_images_avg_nl[f]:
        img, year, loc, actmap = data
        C = 0 if year < 2012 else 1
        img = img[:, :, C]
        imgs.append(img)
        locs.append(loc)
        actmaps.append(actmap)

    title = f'Filter {f}'
    plot_activations(imgs, actmaps, locs=locs, title=title, size=2, nl=True)

# DHS MS

In [ ]:
actmaps_pkl_path = 'dhs_incountryD_actmaps_ms.pkl'
if not os.path.exists(actmaps_pkl_path):
    ckpt_path = os.path.join(CKPTS_DIR, *MODEL_PATHS['incountry_resnet_ms_D'])
    fold = 'D'
    ls_bands = 'ms'
    nl_band = None
    tensor_name = 'scale3_img'

    top_images_avg_ms = get_max_act_images(ckpt_path, fold, ls_bands, nl_band, tensor_name)
    with open(actmaps_pkl_path, 'wb') as f:
        pickle.dump(top_images_avg_ms, f)

In [ ]:
with open(actmaps_pkl_path, 'rb') as f:
    top_images_avg_ms = pickle.load(f)

In [ ]:
def plot_dhs_ms_images(top_images_avg_ms, filters_to_label=None):
    '''
    Args
    - top_images_avg: dict, filter number (int) => list of (value, data) tuples
      - value = (mean filter activation for a particular image, negative image index)
      - data = (img, year1, year2, loc, label, pred, activation map)
    - filters_to_label: dict, filter number (int) => filter label
      - optional: set to None to plot activation maps for all filters
    '''
    for f in range(len(top_images_avg_ms)):
        savedir = None
        filter_label = None
        if filters_to_label is not None:
            if f not in filters_to_label:
                continue
            filter_label = filters_to_label[f]
            savedir = os.path.join(
                LOGS_DIR, MODEL_PATHS['incountry_resnet_ms_D'][0], 'actmaps', str(f))

        imgs = []
        years = []
        locs = []
        actmaps = []

        for value, data in top_images_avg_ms[f]:
            img, year, loc, actmap = data
            img = img[:, :, [2, 1, 0]]  # convert from BGR to RGB
            imgs.append(img)
            years.append(year)
            locs.append(loc)
            actmaps.append(actmap)

        title = f'Filter {f:d}'
        if filter_label is not None:
            title += f': {filter_label}'
        plot_activations(imgs, actmaps, locs=locs, title=title, savedir=savedir)

In [ ]:
# plot_dhs_ms_images(top_images_avg_ms, filters_to_label=None)

In [ ]:
ms_filters_to_label = {
    2: 'urban',
    10: 'water',
    20: 'orange',
    45: 'water',
    59: 'senegal_river',
    64: 'farmland',
    79: 'canyons',
    82: 'greenery',
    102: 'greenery',
    105: 'orange'
}

plot_dhs_ms_images(top_images_avg_ms, filters_to_label=ms_filters_to_label)

In [ ]:
for f in [2, 10, 64]:
    savedir = os.path.join(LOGS_DIR, MODEL_PATHS['incountry_resnet_ms_D'][0], 'actmaps', str(f))
    top_row, bot_row = [], []
    for i in range(NUM_TOP_IMGS):
        top_row.append(PIL.Image.open(os.path.join(savedir, f'img_{i}.png')))
        bot_row.append(PIL.Image.open(os.path.join(savedir, f'actmap_{i}.png')))
    imgs = [top_row, bot_row]
    grid = images_grid(imgs, size=224, spacing=5, color=255)
    savepath = os.path.join(savedir, 'grid.png')
    PIL.Image.fromarray(grid).save(savepath, optimize=True)

# LSMS indexofdelta MS

In [ ]:
def get_lsmsdelta_test_batcher(ls_bands, nl_band, indexofdelta=False):
    '''
    Args
    - ls_bands: one of ['rgb', 'ms', None]
    - nl_band: one of ['merge', 'split', None]
    - indexofdelta: False for delta-of-index, True for index-of-delta

    Returns
    - batcher: DeltaBatcher
    '''
    delta_pairs_df = pd.read_csv('../data/lsmsdelta_pairs.csv')

    if indexofdelta:
        label_col = 'index_diff'
    else:
        label_col = 'index'

    # split => np.array
    tfrecord_pairs, households, labels = delta_batcher.get_lsms_tfrecord_pairs(
        indices_dict=None,  # get all TFRecords
        delta_pairs_df=delta_pairs_df,
        index_cols=['tfrecords_index.x', 'tfrecords_index.y'],
        other_cols=['x', label_col])
    weights = households / np.sum(households) * len(households)

    extra_fields = {'labels': labels, 'weights': weights}

    b = delta_batcher.DeltaBatcher(
        tfrecord_pairs=tfrecord_pairs,
        dataset='LSMS',
        batch_size=BATCH_SIZE,
        label_name=None,
        num_threads=4,
        epochs=1,
        ls_bands=ls_bands,
        nl_band=nl_band,
        shuffle=False,
        augment='none',
        normalize=True,
        cache=False,
        orig_labels=None,
        extra_fields=extra_fields)
    return b

In [ ]:
def run_lsmsdelta_batches(sess, tensors_dict_ops, tensor_name, max_nbatches=None, verbose=False):
    '''Runs the ops in tensors_dict_ops for a fixed number of batches or until
    reaching a tf.errors.OutOfRangeError, concatenating the runs.

    Note: assumes that the dataset iterator doesn't need initialization, or is
        already initialized.

    Args
    - sess: tf.Session
    - tensors_dict_ops: dict, str => tf.Tensor, shape [batch_size] or [batch_size, D]
      - keys: ['images', 'years1', 'years2', 'locs', tensor_name]
    - tensor_name: str, key in TENSOR_NAMES
    - max_nbatches: int, maximum number of batches to run the ops for,
        set to None to run until reaching a tf.errors.OutOfRangeError
    - verbose: bool, whether to output batch processing progress

    Returns
    - top_images_avg: dict, filter number (int) => list of (value, data) tuples
        value = (mean filter activation for a particular image, negative image index)
        data = (img, year1, year2, loc, label, pred, activation map)
    '''
    top_images_avg = defaultdict(list)

    curr_batch = 0
    start_time = time.time()
    try:
        while True:
            all_tensors = sess.run(tensors_dict_ops)

            imgs = all_tensors['images']  # N, H, W, C
            years1 = all_tensors['years1']
            years2 = all_tensors['years2']
            locs = all_tensors['locs']
            labels = all_tensors['labels']
            preds = all_tensors['preds']
            actmaps = all_tensors[tensor_name]

            batch_size, _, _, num_filters = actmaps.shape

            actmaps_means = np.mean(actmaps, axis=(1, 2), dtype=np.float64)  # shape [batch_size, num_filters]

            for i in range(batch_size):
                img = imgs[i]
                y1 = years1[i]
                y2 = years2[i]
                loc = tuple(locs[i])
                label = labels[i]
                pred = preds[i]
                actmap = actmaps[i]  # shape [H, W, num_filters]
                actmap_means = actmaps_means[i]  # shape [num_filters]

                for f in range(num_filters):
                    value = (actmap_means[f], -(i + curr_batch*BATCH_SIZE))
                    data = (img, y1, y2, loc, label, pred, actmap[:, :, f])
                    add_to_heap(h=top_images_avg[f], k=NUM_TOP_IMGS, value=value, data=data)

            curr_batch += 1
            if verbose:
                speed = curr_batch / (time.time() - start_time)
                print(f'\rRan {curr_batch} batches ({speed:.3f} batch/s)', end='')
            if (max_nbatches is not None) and (curr_batch >= max_nbatches):
                break
            if curr_batch >= 15:
                break

    except tf.errors.OutOfRangeError:
        pass

    print()  # print a newline, since the previous print()'s don't print newlines
    return top_images_avg

In [ ]:
def get_lsmsdelta_max_act_images(ckpt_path: str, fold: str, ls_bands: str,
                                 nl_band: str, tensor_name: str, indexofdelta=False):
    '''
    Args
    - ckpt_path: str
    - fold: str, one of ['A', 'B', 'C', 'D', 'E']
    - ls_bands: str or None
    - nl_band: str or None
    - tensor_name: str, key of TENSOR_NAMES
    - indexofdelta: bool

    Returns
    - top_images_avg: dict, filter number (int) => list of (value, data) tuples
        value = (mean filter activation for a particular image, negative image index)
        data = (img, year1, year2, loc, label, pred, activation map)
    '''
    tf.reset_default_graph()

    print('=== Running model ===')
    print('- ckpt:', ckpt_path)
    print('- fold:', fold)
    print('- ls_bands, nl_band:', ls_bands, nl_band)

    batcher = get_lsmsdelta_test_batcher(ls_bands, nl_band, indexofdelta)
    init_iter, batch_op = batcher.get_batch()

    print('Building model...')
    model = Hyperspectral_Resnet(batch_op['images'], is_training=IS_TRAINING, **MODEL_PARAMS)
    preds = tf.squeeze(model.outputs, name='preds')

    graph = tf.get_default_graph()
    tensors_dict_ops = {
        'images': batch_op['images'],
        'years1': batch_op['years1'],
        'years2': batch_op['years2'],
        'locs': batch_op['locs'],
        'labels': batch_op['labels'],
        'preds': preds,
        tensor_name: graph.get_tensor_by_name(TENSOR_NAMES[tensor_name]),
    }
    # note: these are not the cross-validated ridge-regression-tuned preds

    print('Creating session...')
    os.environ['CUDA_VISIBLE_DEVICES'] = str(GPU)
    config_proto = tf.ConfigProto()
    config_proto.gpu_options.per_process_gpu_memory_fraction = GPU_USAGE

    with tf.Session(config=config_proto) as sess:
        sess.run(init_iter)

        # clear the model weights, then load saved checkpoint
        print('Loading saved ckpt...')
        saver = tf.train.Saver(var_list=None)
        sess.run([tf.global_variables_initializer(), tf.local_variables_initializer()])
        saver.restore(sess, ckpt_path)

        # run the saved model
        top_images_avg = run_lsmsdelta_batches(sess, tensors_dict_ops, tensor_name, verbose=True)
    return top_images_avg

In [ ]:
def plot_delta_activations(imgs, actmaps, locs=None, years=None,
                           labels=None, preds=None,
                           title=None, size=4, nl=False, savedir=None):
    '''
    Args
    - imgs: list of (img1, img2) tuples, length N
      - imgX: np.array, shape [H, W] or [H, W, 3]
      - if [H, W, 3], then band order is assumed to be R, G, B
    - actmaps: list of np.array, length N, each np.array has shape [actH, actW]
    - locs: list of (lat, lon) tuples, length N
    - years: list of (year1, year2) tuples, length N
    - labels: list of float, length N
    - preds: list of float, length N
    - title: str, figure title
    - size: int, size of each img, in inches
    - nl: bool, when plotting nightlights images
    - savedir: str, path to directory to save imgs and activation maps
    '''
    nimgs = len(imgs)
    assert len(actmaps) == nimgs

    # make copy, so we don't modify the originals
    np_imgs = np.array(imgs)
    np_actmaps = np.abs(np.array(actmaps))  # activation maps aren't always after a ReLU

    # sort images by mean activation in descending order
    sorted_index = np.argsort(np.mean(np_actmaps, axis=(1, 2)))[::-1]
    np_imgs = np_imgs[sorted_index]
    np_actmaps = np_actmaps[sorted_index]

    if locs is not None:
        locs = np.array(locs)[sorted_index]  # make copy
    if years is not None:
        years = np.array(years)[sorted_index]  # make copy
    if labels is not None:
        labels = np.array(labels)[sorted_index]  # make copy
    if preds is not None:
        preds = np.array(preds)[sorted_index]  # make copy

    max_act = np.percentile(np_actmaps, q=99)

    # row 1 = img1, row 2 = img2, row3 = actmap
    fig, axs = plt.subplots(3, nimgs, figsize=[size*nimgs, size*3])

    for i in range(nimgs):
        img1 = smart_img_rescale(np_imgs[i][0])
        img2 = smart_img_rescale(np_imgs[i][1])
        actmap = np_actmaps[i]

        if nl:
            mean = np.mean(actmap)
            std = np.std(actmap)
            actmap_max = min(np.max(actmap), mean + 6 * std)
            actmap_min = max(np.min(actmap), mean - 6 * std)
            actmap = (actmap - actmap_min) / actmap_max
            actmap = np.clip(actmap, a_min=0, a_max=1)
        else:
            actmap = np.clip(actmap, a_min=0, a_max=max_act) / max_act
    
        # origin='lower' to match lat/lon direction
        axs[0, i].imshow(img1, origin='lower', vmin=0, vmax=1)
        axs[1, i].imshow(img2, origin='lower', vmin=0, vmax=1)
        axs[2, i].imshow(actmap, origin='lower', vmin=0, vmax=1,
                         interpolation='none', cmap='gray')

        for j in range(3):
            axs[j, i].axis('off')

        ax_title = []
        if locs is not None:
            lat, lon = locs[i]
            ax_title.append(f'loc: ({lat:.4f}, {lon:.4f})')
        if years is not None:
            (y1, y2) = years[i]
            ax_title.append(f'y: {y1}-{y2}')
        if labels is not None:
            label = labels[i]
            ax_title.append(f'\nlabel:{label:.2f}')
        if preds is not None:
            pred = preds[i]
            ax_title.append(f'pred:{pred:.2f}')
        if len(ax_title) > 0:
            ax_title = ' '.join(ax_title)
            axs[0, i].set_title(ax_title)

        if savedir is not None:
            os.makedirs(savedir, exist_ok=True)
            img1_filename = os.path.join(savedir, f'img_{i}_1.png')
            print('Saving image to', img1_filename)
            # plt.imsave(img_filename, img, vmin=0, vmax=1, format='png', origin='lower')
            PIL.Image.fromarray((img1 * 255).astype(np.uint8)).transpose(PIL.Image.FLIP_TOP_BOTTOM).save(img1_filename, optimize=True)

            img2_filename = os.path.join(savedir, f'img_{i}_2.png')
            print('Saving image to', img2_filename)
            PIL.Image.fromarray((img1 * 255).astype(np.uint8)).transpose(PIL.Image.FLIP_TOP_BOTTOM).save(img2_filename, optimize=True)

            actmap_filename = os.path.join(savedir, f'actmap_{i}.png')
            print('Saving activation map to', actmap_filename)
            # plt.imsave(actmap_filename, actmap, vmin=0, vmax=1, format='png', origin='lower', cmap='gray')
            PIL.Image.fromarray((actmap * 255).astype(np.uint8)).transpose(PIL.Image.FLIP_TOP_BOTTOM).save(actmap_filename, optimize=True)

    if title is not None:
        fig.suptitle(title, y=1.03)
    fig.subplots_adjust(wspace=0, hspace=0)
    fig.tight_layout()
    plt.show()

In [ ]:
actmaps_pkl_path = 'lsms_indexofdelta_incountry_ms_D_actmaps.pkl'
if not os.path.exists(actmaps_pkl_path):
    ckpt_path = os.path.join(CKPTS_DIR, *MODEL_PATHS['lsms_indexofdelta_incountry_ms_D'])
    fold = 'D'
    ls_bands = 'ms'
    nl_band = None
    tensor_name = 'scale3_img'

    top_images_avg_ms = get_lsmsdelta_max_act_images(
        ckpt_path, fold, ls_bands, nl_band, tensor_name)
    with open(actmaps_pkl_path, 'wb') as f:
        pickle.dump(top_images_avg_ms, f)

In [ ]:
with open(actmaps_pkl_path, 'rb') as f:
    top_images_avg_ms = pickle.load(f)

In [ ]:
def plot_lsmsdelta_images(top_images_avg_ms, filters_to_label=None):
    '''
    Args
    - top_images_avg: dict, filter number (int) => list of (value, data) tuples
      - value = (mean filter activation for a particular image, negative image index)
      - data = (img, year1, year2, loc, label, pred, activation map)
    - filters_to_label: dict, filter number (int) => filter label
      - optional: set to None to plot activation maps for all filters
    '''
    for f in sorted(top_images_avg_ms.keys()):
        savedir = None
        filter_label = None
        if filters_to_label is not None:
            if f not in filters_to_label:
                continue
            filter_label = filters_to_label[f]
            savedir = os.path.join(
                LOGS_DIR, MODEL_PATHS['lsms_indexofdelta_incountry_ms_D'][0], 'actmaps', str(f))

        imgs = []
        years = []
        locs = []
        labels = []
        preds = []
        actmaps = []

        for value, data in top_images_avg_ms[f]:
            img, y1, y2, loc, label, pred, actmap = data
            assert img.shape[2] % 2 == 0
            C = int(img.shape[2] / 2)
            img1 = img[:, :, [2, 1, 0]]  # convert from BGR to RGB
            img2 = img[:, :, [C+2, C+1, C]]
            imgs.append((img1, img2))
            years.append((y1, y2))
            locs.append(loc)
            labels.append(label)
            preds.append(pred)
            actmaps.append(actmap)

        title = f'Filter {f}'
        if filter_label is not None:
            title += f': {filter_label}'
        plot_delta_activations(imgs, actmaps, years=years, locs=locs, labels=labels,
                               preds=preds, title=title, savedir=savedir)

In [ ]:
# plot_lsmsdelta_images(top_images_avg_ms, filters_to_label=None)

In [ ]:
ms_filters_to_label = {
    8: 'water',
    23: 'greenery',
    25: 'missing values',
    59: 'urban centers',
    84: 'negative preds',
    86: 'purple year1',
    87: 'negative greenery',
    94: 'urban expansion',
    98: 'drying up',
    117: 'urban expansion',
    118: 'increased image clarity in year2',
}

plot_lsmsdelta_images(top_images_avg_ms, filters_to_label=ms_filters_to_label)

Complete List of Filter Labels

- 6: missing values in y1
- 8: water
- 9: missing values in y1
- 16: missing values in y1
- 17: missing values in y1
- 19: missing values in y1
- 20: water
- 23: greenery
- 25: missing values in y1
- 48: missing values in y1
- 56: water
- 59: urban centers
- 61: missing values in y1
- 66: missing values in y1
- 75: missing values in y1
- 79: missing values in y1
- 84: negative preds, although not necessarily negative labels
- 86: purple y1
- 87: negative green stuff
- 88: positive preds on yellow stuff, although not necessarily positive labels
- 91: water
- 92: missing values in y1
- 94: urban expansion (maybe?)
- 96: missing values in y1
- 98: drying up (maybe?)
- 117: urban expansion
- 118: increased clarity in y2